In [1]:
import numpy as np
import pydot
from IPython.display import HTML, SVG, display
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         FindResourceOrThrow, GenerateHtml,
                         InverseDynamicsController, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant, Parser,
                         Simulator, StartMeshcat)

from manipulation import running_as_notebook

In [2]:
#start the visualizer
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
filename = "drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf"
def create_builder():
    #create a diagram builder object
    builder = DiagramBuilder()
    return builder
builder = create_builder()

def create_plant_scene_graph(builder):
    #plant and scene graph pointers for the newly created multibody plant and scene graph returned
    #they are added to the diagram
    #Makes a new MultibodyPlant with discrete update period time_step and adds it to a diagram builder together with the provided SceneGraph instance, connecting the geometry ports.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)    
    return plant, scene_graph
plant, scene_graph = create_plant_scene_graph(builder)


def create_iiwa_model(plant, scene_graph, filename):
    iiwa_model = Parser(plant, scene_graph).AddModelFromFile(FindResourceOrThrow(filename))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_link_0"))
    plant.Finalize()
    return iiwa_model


iiwa_model = create_iiwa_model(plant, scene_graph, filename)


position_vector = [-1.57, 0.1, 0, -1.2, 0, 1.6, 0]
    

In [4]:
def add_meshcat2builder(builder, scene_graph, meshcat):
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    return visualizer

visualizer = add_meshcat2builder(builder, scene_graph, meshcat)

In [5]:
def iiwa_position_set(builder, plant, position_vector):    
    #create a diagram out of the wired systems
    diagram = builder.Build()    
    #extract context from the diagram
    context = diagram.CreateDefaultContext()
    #extract plant context from the full context
    plant_context = plant.GetMyMutableContextFromRoot(context)
    q0 = q0 = np.array(position_vector)
    x0 = np.hstack((q0, 0*q0))
    plant.SetPositions(plant_context, q0)
    #iiwa_controller.GetInputPort('desired_state').FixValue(iiwa_controller.GetMyMutableContextFromRoot(context), x0)
    plant.get_actuation_input_port().FixValue(plant_context, np.zeros(7))
    return diagram,context

diagram,context = iiwa_position_set(builder, plant, position_vector)

In [5]:
def iiwa_controller(builder, plant):
    Kp = np.full(7, 10)
    Ki = 2 * np.sqrt(Kp)
    Kd = np.full(7, 1)
    iiwa_controller = builder.AddSystem(InverseDynamicsController(plant, Kp, Ki, Kd, False))
    iiwa_controller.set_name("iiwa_controller");
    #create feedback loop
    #plant state output to controller input
    #controller output to plant actuation input
    builder.Connect(plant.get_state_output_port(iiwa_model),
                    iiwa_controller.get_input_port_estimated_state())
    builder.Connect(iiwa_controller.get_output_port_control(),
                    plant.get_actuation_input_port())
    
    return iiwa_controller

iiwa_controller = iiwa_controller(builder, plant)

In [6]:
def iiwa_position_set(builder, plant, iiwa_controller, position_vector):    
    #create a diagram out of the wired systems
    diagram = builder.Build()    
    #extract context from the diagram
    context = diagram.CreateDefaultContext()
    #extract plant context from the full context
    plant_context = plant.GetMyMutableContextFromRoot(context)
    q0 = q0 = np.array(position_vector)
    x0 = np.hstack((q0, 0*q0))
    plant.SetPositions(plant_context, q0)
    iiwa_controller.GetInputPort('desired_state').FixValue(iiwa_controller.GetMyMutableContextFromRoot(context), x0)
    #plant.get_actuation_input_port().FixValue(plant_context, np.zeros(7))
    return diagram,context

diagram,context = iiwa_position_set(builder, plant, iiwa_controller, position_vector)

In [7]:
def iiwa_animate(visualizer,diagram, context):
    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)
    visualizer.StartRecording()
    simulator.AdvanceTo(20.0)
    visualizer.StopRecording()
    visualizer.PublishRecording()
    
iiwa_animate(visualizer,diagram, context)